### Умные указатели (C++11)

https://en.cppreference.com/w/cpp/memory/unique_ptr

https://en.cppreference.com/w/cpp/memory/shared_ptr

https://en.cppreference.com/w/cpp/memory/weak_ptr

https://en.cppreference.com/w/cpp/memory/unique_ptr/make_unique

https://en.cppreference.com/w/cpp/memory/shared_ptr/make_shared

**Доклады**:
* [Back to Basics: Smart Pointers - Rainer Grimm - CppCon 2020](https://www.youtube.com/watch?v=sQCSX7vmmKY)

<br />

##### Мотивацонные примеры, зачем нужны умные указатели:

**Вариант 1.** Мы пишем своих angry birds. Нужно уметь подгружать для игрушки ресурсы по сети.

```c++
class NetworkSystem
{
public:
    virtual ~NetworkSystem() = default;

    virtual std::string load(const std::string& url) = 0;
};

class GameResourcesLoader
{
public:
    GameResourcesLoader(NetworkSystem* network) {...}

private:
    NetworkSystem* network_;
};

class PromoEventsLoader
{
public:
    PromoEventsLoader(NetworkSystem* network) {...}

private:
    NetworkSystem* network_;
};
```

В чём проблема?

<br />

**Вариант 2.** Мы реализуем первое домашнее задание и решили повыпендриваться с интерфейсами хешеров:

```c++
class Hasher
{
public:
    virtual ~Hasher() = default;

    virtual std::uint32_t calc_hash(std::istream& is) = 0;
};

Hasher *create_hasher(HasherType type)
{
    if (type == CRC32)
        return new HasherCRC32();
    else
        return new HasherSUM32();
}

bool run(const std::string& filename, HasherType type)
{
    std::ifstream ifs(filename);
    if (!ifs)
        return false;

    Hasher *hasher = create_hasher(type);

    std::cout << hasher->calc_hash(ifs);

    delete hasher;
}
```

В чём проблема?

<br />

**Вариант 3.** Посложнее. В чём проблема здесь?

```c++
// Интерфейс работы с кодировкой видеоданных.
class VideoCodec { ... }

class VideoDecoder
{
public:
    VideoDecoder()
        : codecs_{
            new VideoCodecDIVX(),
            new VideoCodecX264(),
            new VideoCodecXVID()
        }
    {}

    ~VideoDecoder() {
        for (auto* codec: codecs_)
            delete codec;
    }
    
    ...;

private:
    std::vector<VideoCodec*> codecs_;
};
```

<br />

##### Варианты умных (и не очень) указателей

Эти правила появились с С++11, и отчасти являются следствием реализации умных указетелей, отчасти - договорённости. Те места, где правила нарушаются, должны быть в коде хорошо откомментированы.

* `unique_ptr` - указатель, описывающий уникальное владение объектом
* `shared_ptr` - указатель, описывающий множественное владние объектом
* `weak_ptr` - указатель для отслеживания времени жизни shared-объектов
* `raw pointer` - невладеющий указатель (`Item*`)

__Замечание__: за исключением задокументированных случаев сырой указатель - невладеющий, код, работающий с сырым указателем, не должен вызывать `free` и `delete`. Т.е.:

* метод возвращает невладеющий указатель, ответственность за удаление объекта лежит на классе

```c++
class ItemsStorage
{
    ...
    Item* find(int key) const;
};
```

* функция возвращает невладеющий указатель, она знает, что вызывающий код не должен управлять временем жизни объекта

```c++
const char* get_name()
{
    return "Dobryna Nikitich";
}
```

* функция принимает невладеющий указатель, она не в праве управлять временем жизни объекта. Семантика указателя: объект может отсутствовать.

```c++
Home build_home(const Roof* roof);
```

* функция принимает невладеющий указатель и возвращает невладеющий указатель:

```c++
const char* find_char(const char* str, char c);
```

* особый случай - функции `malloc` && `free`:

```c++
void* malloc(size_t size);
free(void *);
```

<br />

##### std::unique_ptr IR

Упрощённый вариант для демонстрации

```c++
template<typename T>
class unique_ptr
{
public:
    unique_ptr() : ptr(nullptr) {}
    
    unique_ptr(T* p) : ptr(p) {}
    
    ~unique_ptr()
    {
        delete ptr;
    }
    
    // указатель уникальный - копировать его нельзя
    unique_ptr(const unique_ptr&) = delete;
    unique_ptr& operator = (const unique_ptr&) = delete;
    
    // а перемещать из одного объекта в другой - можно:
    unique_ptr(unique_ptr&& rhs) noexcept
    {
        ptr = rhs.ptr;
        rhs.ptr = nullptr;  // 1. ~unique_ptr отработает нормально?
                            // 2. nullptr - специфицирован!
    }
    unique_ptr& operator = (unique_ptr&& rhs) noexcept
    {
        if (this != &rhs)
        {
            delete ptr;

            ptr = rhs.ptr;
            rhs.ptr = nullptr;  
        }
        return *this;
    }
    
    T& operator ->() const noexcept
    {
        return *ptr;
    }    

private:
    T* ptr;
};
```

Чему равен `sizeof` такого `unique_ptr` ?

<br />

#### std::make_unique

Пример:

```c++
std::unique_ptr<Person> create_person(std::unique_ptr<Head> head,
                                      std::unique_ptr<Body> body);

create_person(new Head, new Body);
```

Напомните, в чём проблема?

<br />

Теперь мы грамотные и можем исправить проблему:

```c++
create_person(std::unique_ptr<Head>(new Head),
              std::unique_ptr<Body>(new Body));
```

Почему проблема до сих пор с нами?

<br />

Корректное решение:

```c++
create_person(std::make_unique<Head>(),
              std::make_unique<Body>());
```

**Замечание**: Проблема с утечкой в коде

```c++
create_person(std::unique_ptr<Head>(new Head),
              std::unique_ptr<Body>(new Body));
```

Актуальна только до С++14. В стандарте С++17 скорректировали правила порядка вычислений аргументов ([пункт 15](https://en.cppreference.com/w/cpp/language/eval_order)), и теперь аргумент функции обязан быть вычислен полностью до вычисления другого аргумента, но порядок вычисления аргументов всё равно не определён.

```c++
// undefined   behavior until C++17
// unspecified behavior since C++17
int i = 0;
f(++i, ++i);
```

<br />

`std::make_unique` умеет передавать аргументы в конструктор:

```c++
class Head {
public:
    Head(Color eyes_color, float nose_length_cm);
    ...;
};

std::unique_ptr<Head> buratino_head = std::make_unique<Head>(Color::Blue, 25);
```

<br />

##### Как пользоваться std::unique_ptr

**Пример 1** для angry birds

```c++
// Интерфейс работы с сетью в приложении.
class INetwork { ... }

// Интерфейс доступа до ресурсов игры.
class IResources { ... }

// Контекст игровой сессии.
class SessionContext {
public:
    SessionContext(std::unique_ptr<INetwork> network,
                   std::unique_ptr<IResources> resources)
        : network_(std::move(network))
        , resources_(std::move(resources))
    {}
    
    // доступ до актуальной реализации сети
    INetwork& network() const { return *network_; }
    
    // доступ до актуального хранилища ресурсов
    IResources& resources() const { return *resources_; }
    
private:
    std::unique_ptr<INetwork> network_;
    std::unique_ptr<IResources> resources_;
};

// Создаём контекст для игрока.
SessionContext makeSessionContextForGame() {
    return SessionContext(
        std::make_unique<NetworkImpl>(),
        std::make_unique<ResourcesImpl>());
}

// Создаём контекст для тестирования.
SessionContext makeSessionContextForTesting() {
    return SessionContext(
        std::make_unique<NetworkMock>(),
        std::make_unique<ResourcesMock>());
}

// Создаём контекст для презентации заказчику.
SessionContext makeSessionContextForPresentation() {
    return SessionContext(
        std::make_unique<SuperFastNetwork>(),
        std::make_unique<CachedResources>());
}


void use(SessionContext& context) {
    INetwork& network = context.network();    
    network.load();
}
```

**Пример 2** для hasher

```c++
class Hasher
{
public:
    virtual ~Hasher() = default;
    virtual std::uint32_t calc_hash(std::istream& is) = 0;
};

std::unique_ptr<Hasher> create_hasher(HasherType type)
{
    if (type == CRC32)
        return std::make_unique<HasherCRC32>();
    else
        return std::make_unique<HasherSUM32>();
}

bool run(const std::string& filename, HasherType type)
{
    std::ifstream ifs(filename);
    if (!ifs)
        return false;

    std::unique_ptr<Hasher> hasher = create_hasher(type);
    std::cout << hasher->calc_hash(ifs);
}
```

**Пример 3:** для набора кодеков

```c++
// Интерфейс работы с кодировкой видеоданных.
class VideoCodec { ... }

class VideoDecoder
{
public:
    VideoDecoder()
    {
        codecs_.reserve(3);
        codecs_.emplace_back(std::make_unique<VideoCodecDIVX>());
        codecs_.emplace_back(std::make_unique<VideoCodecX264>());
        codecs_.emplace_back(std::make_unique<VideoCodecXVID>());    
    }

    ...;

private:
    std::vector<std::unique_ptr<VideoCodec>> codecs_;
};

```

<br />

##### std::shared_ptr IR

Перед тем как показать ответ, попробуйте ответить на следующие вопросы:

Объект должен умереть, тогда и только тогда когда умрёт __последний__ его владелец. Какую информацию придётся хранить?

Где эту информацию хранить? Как насчёт такого примера:

```c++
std::shared_ptr<T> a(new T);
std::shared_ptr<T> b = a;
std::shared_ptr<T> c = b;

a.reset();
b.reset();

// объект должен жить, т.к. им владеют через c

```

<br />

Ответ:

![](sp_internals.PNG)

<br />

##### Как работает shared_ptr

Потренируемся на кошках. Эти примеры надо рисовать на доске (что происходит в памяти), так понимают лучше

```c++
{
    std::shared_ptr<Cat> cat1(new Cat);

    std::shared_ptr<Cat> cat2;

    cat2 = cat1;

    std::shared_ptr<Cat> cat3 = cat1;

    cat1.reset();

    std::weak_ptr<Cat> weak_cat = cat2;

    std::shared_ptr<Cat> cat4 = weak_cat.lock();
    if (cat4)
        ...;
}
```

<br />

##### Специфические ошибки с shared_ptr

* Один объект - несколько кластеров `shared_ptr`-ов

    ```c++
    {
        Cat* y = new Cat();
        std::shared_ptr<Cat> cat1(y);
        std::shared_ptr<Cat> cat2(y);
    }
    ```

* циклы владения

```c++
class Cat {
public:
    void set_frient(std::shared_ptr<Cat> f) {
        friend_ = std::move(f);
    }

private:
    std::shared_ptr<Cat> friend_;
};

{
    auto barsic = std::shared_ptr<Cat>(new Cat);
    auto marsic = std::shared_ptr<Cat>(new Cat);
    barsic->set_friend(marsic);
    marsic->set_friend(barsic);
}
```

<br />

##### make_shared - способ быстрее создавать shared_ptr

https://arne-mertz.de/2018/09/make_shared-vs-the-normal-shared_ptr-constructor/

```c++
std::vector<std::shared_ptr<Cat>> herd;
herd.reserve(100);
for (int i = 0; i < 100; ++i)
    herd.push_back(std::shared_ptr<Cat>(new Cat()));
```

![](shared_ptr_naive.png)

200 аллокаций на объекты + 1 аллокация на вектор

<br />

```c++
std::vector<std::shared_ptr<Cat>> herd;
herd.reserve(100);
for (int i = 0; i < 100; ++i)
    herd.push_back(std::make_shared<Cat>());
```

![](shared_ptr_make_shared.png)

100 аллокаций на объекты + 1 аллокация на вектор

<br />

##### custom deleter && scope exit

Умным указателям можно задавать пользовательские deleter-ы.

Может быть полезно для:
1. управления не-RAII - объектами
2. донастройки разрушения объекта
3. отлавливания момента, когда объект разрушен

Пример 1: управление не-RAII-объектами.
        
```c++
void write_to_csv(const char* filename)
{
    FILE* fo = fopen(filename, "w");
    // код записи данных в файл тут
    // TODO: в чём потенциальная проблема такой реализации?
    fclose(fo);
}
```

```c++
void write_to_csv(const char* filename)
{
    std::unique_ptr<FILE, int (*)(FILE*)> fo(fopen(filename, "w"),
                                             &fclose);
    // код записи данных в файл тут
}
```

Как это читать смертным:

```c++
// уникальное владение ресурсом
std::unique_ptr<
                // ресурс - FILE, внтури хранится FILE*
                FILE,
                // тип нестандартного удалятора (в данном случае - сигнатура функции, т.к. передаётся функция)
                int (*)(FILE*)>
                // имя переменной + вызов конструктора
                fo(
                    // ресурс, которым владеем
                    fopen(filename, "r"),
                    // нестандартный удалятор - указатель на функцию
                    &fclose);
```

Вариации:

```c++
// изначальный вариант
std::unique_ptr<FILE, int (*)(FILE*)> p(fopen("input.txt", "r"),
                                        &fclose);

// если лень вручную вспоминать тип удалятора
std::unique_ptr<FILE, decltype(&fclose)> p(fopen("input.txt", "r"),
                                           &fclose);

// через deleter functor
struct FileCloser {  
    void operator()(FILE* f) {
        fclose(f);
    }
};

std::unique_ptr<FILE, FileCloser> p(fopen("input.txt", "r"),
                                    FileCloser());

// через лямбду
auto close_file = [](FILE* f) { fclose(f); };
std::unique_ptr<FILE, decltype(close_file)> p(fopen("input.txt", "r"),
                                              close_file);

// shared_ptr дороже в runtime, но приятнее конструировать,
//            не нужно прямо указывать тип deleter
std::shared_ptr<FILE> p(fopen("input.txt", "r"), close_file);
```

Пример 2: донастройка разрушения объекта

```c++
std::shared_ptr<FILE> p(fopen("error.log", "r"),
                        [](FILE* f) {
                            std::cout << "you are going to die!\n";
                            fclose(f);
                            
                            schedule_send_error_log_to_sever("error.log");
                        });
```

__Вопрос:__ чего не должны делать custom deleter?

<details>
<summary>Подсказка</summary>
<p>

Бросать исключений!

</p>
</details>

<br />

#### Правила работы с уникальными указателями:

* Владение объектом:
  * по значению
  * через `std::unique_ptr` в случае единоличного владения
  * через `std::shared_ptr` в случае множественного владения
  
  Не надо так!
  ```c++
  struct Point
  {
      std::unique_ptr<float> x;
      std::unique_ptr<float> y;
  };
  ```
  
  Будьте человечнее:
  ```c++
  struct Point
  {
      float x;
      float y;
  };
  ```
  
* В нормальном коде почти никогда не нужно явно вызывать `new`/`delete`. Для этого есть `std::make_unique` и `std::make_shared`:
  * `make_unique` помогает избежать проблем с утечкой при исключениях
  * `make_shared` дополнительно работает быстрее конструктора `shared_ptr`, т.к. выполняет меньше аллокаций

* Для разрыва циклов владения используйте `std::weak_ptr`

* Не надо без нужды передавать в функцию умный указатель - передавайте сам объект. Умный указатель передаётся только в том случае, если функция работает с владением объекта:

    Случай, когда функции не должны работать со способом владения объектом:
    ```c++
    void print_person(const Person& person) {
        std::cout << person.name << ' ' << person.surname << ' ' << person.age;
    }

    void make_elder(Person& person) {
        ++person.age;

        if (person.age == 33)
            std::cout << "time to fight!" << std::endl;
    }

    std::shared_ptr<Person> ilya = std::make_shared<Person>("Ilya", "Muromec", 32);
    print_person(*ilya);
    make_elder(*ilya);
    ```
    
    Случай, когда должны:
    ```c++
    class Squad {
    public:
        void add_warrior(std::shared_ptr<Person> warrior) {
            warriors_.push_back(warrior);
        }
        
    private:
        std::vector<std::shared_ptr<Person>> warriors_;
    };
    
    Squad s;
    std::shared_ptr<Person> ilya = std::make_shared<Person>("Ilya", "Muromec", 32);
    s.add_warrior(ilya);
    ```

<br />

За пределами темы:

* custom allocator
* `shared_from_this`
* специальные `static_pointer_cast` / `dynamic_pointer_cast` для умных указателей
* `std::shared_ptr` to class members
* intrusive pointers